In [5]:
import pandas as pd
from pathlib import Path

beh_path = Path("../data/поведенческие_паттерны_клиентов_3.csv")
tx_path  = Path("../data/транзакции_в_Мобильном_интернет_Банкинге.csv")

# читаем без заголовков
beh_raw = pd.read_csv(beh_path, sep=";", encoding="cp1251", header=None)
tx_raw  = pd.read_csv(tx_path,  sep=";", encoding="cp1251", header=None)

print("beh_raw:", beh_raw.shape)
print("tx_raw :", tx_raw.shape)

beh_raw.head()


beh_raw: (8589, 19)
tx_raw : (13115, 7)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Дата совершенной транзакции,Уникальный идентификатор клиента,Количество разных версий ОС (os_ver) за послед...,Количество разных моделей телефона (phone_mode...,Модель телефона из самой последней сессии (по ...,Версия ОС из самой последней сессии перед tran...,Количество уникальных логин-сессий (минутных т...,Количество уникальных логин-сессий за последни...,Среднее число логинов в день за последние 7 дн...,Среднее число логинов в день за последние 30 д...,Относительное изменение частоты логинов за 7 д...,Доля логинов за 7 дней от логинов за 30 дней,Средний интервал (в секундах) между соседними ...,Стандартное отклонение интервалов между логина...,Дисперсия интервалов между логинами за 30 дней...,Экспоненциально взвешенное среднее интервалов ...,Показатель “взрывности” логинов: (std?mean)/(s...,Fano-factor интервалов: variance / mean,Z-скор среднего интервала за последние 7 дней ...
1,transdate,cst_dim_id,monthly_os_changes,monthly_phone_model_changes,last_phone_model_categorical,last_os_categorical,logins_last_7_days,logins_last_30_days,login_frequency_7d,login_frequency_30d,freq_change_7d_vs_mean,logins_7d_over_30d_ratio,avg_login_interval_30d,std_login_interval_30d,var_login_interval_30d,ewm_login_interval_7d,burstiness_login_interval,fano_factor_login_interval,zscore_avg_login_interval_7d
2,'2025-03-05 00:00:00.000',453024373,2,2,Vivo V2116,Android/13,6,20,0.8571428571428571,0.6666666666666666,0.2857142857142857,0.3,131460.0,213919.47830901234,45761543200,23007.164409515124,0.23875037020941794,348102.41290126275,-0.22107383756651397
3,'2025-05-19 00:00:00.000',498636442,1,1,Xiaomi_m2006c3mg,Android/10,2,22,0.2857142857142857,0.7333333333333333,-0.6103896103896104,0.09090909090909091,101520.0,82733.72644157334,6844869490.909091,178980.0,-0.101958717043282,67423.85235332044,0.9362566311418639
4,'2025-08-04 00:00:00.000',2934905461,1,1,"iPhone16,1",iOS/18.5,2,2,0.2857142857142857,0.06666666666666667,3.2857142857142856,1.0,67800.0,-1.0,-1.0,67800.0,-1.0,-1.0,-1.0


In [6]:
# поведение
beh = beh_raw.iloc[2:].copy()
beh.columns = beh_raw.iloc[1].tolist()

# транзакции
tx = tx_raw.iloc[2:].copy()
tx.columns = tx_raw.iloc[1].tolist()

print(beh.columns)
print(tx.columns)
beh.head()
tx.head()


Index(['transdate', 'cst_dim_id', 'monthly_os_changes',
       'monthly_phone_model_changes', 'last_phone_model_categorical',
       'last_os_categorical', 'logins_last_7_days', 'logins_last_30_days',
       'login_frequency_7d', 'login_frequency_30d', 'freq_change_7d_vs_mean',
       'logins_7d_over_30d_ratio', 'avg_login_interval_30d',
       'std_login_interval_30d', 'var_login_interval_30d',
       'ewm_login_interval_7d', 'burstiness_login_interval',
       'fano_factor_login_interval', 'zscore_avg_login_interval_7d'],
      dtype='object')
Index(['cst_dim_id', 'transdate', 'transdatetime', 'amount', 'docno',
       'direction', 'target'],
      dtype='object')


,cst_dim_id,transdate,transdatetime,amount,docno,direction,target
2,2937833270,'2025-01-05 00:00:00.000','2025-01-05 16:32:02.000',31000.0,5343,8406e407421ec28bd5f445793ef64fd1,0
3,2096229005,'2025-03-04 00:00:00.000','2025-03-04 17:41:57.000',4000.0,8442,b3a3d4a6006293195d998957d4f01e42,0
4,2937759666,'2025-06-20 00:00:00.000','2025-06-20 10:08:07.000',3000.0,9540,22b84292f0ebce65ad0808342615a03b,0
5,2933493153,'2025-07-06 00:00:00.000','2025-07-06 14:52:13.000',500.0,11685,d677d4e1a0f625e1ad746ea950c9dca9,0
6,456000634,'2024-12-18 00:00:00.000','2024-12-18 14:12:25.000',20000.0,7128,87b698d1edae13c21ce86678de3b8546,0


In [7]:
import numpy as np

def strip_quotes(s: str) -> str:
    s = str(s).strip()
    if s.startswith("'") and s.endswith("'"):
        s = s[1:-1]
    return s

# приводим даты
beh["transdate_dt"] = pd.to_datetime(beh["transdate"].apply(strip_quotes))
tx["transdate_dt"]  = pd.to_datetime(tx["transdate"].apply(strip_quotes))

# cst_dim_id в одну строку
beh["cst_dim_id"] = beh["cst_dim_id"].astype(str)
tx["cst_dim_id"]  = tx["cst_dim_id"].astype(str)


In [8]:
beh_cat_cols = ["cst_dim_id", "last_phone_model_categorical", "last_os_categorical"]
beh_num_cols = [
    c for c in beh.columns 
    if c not in beh_cat_cols + ["transdate", "transdate_dt"]
]

for c in beh_num_cols:
    beh[c] = pd.to_numeric(beh[c], errors="coerce")

# -1.0 считаем "нет данных"
beh[beh_num_cols] = beh[beh_num_cols].replace(-1.0, np.nan)


In [9]:
tx["amount"] = pd.to_numeric(tx["amount"], errors="coerce")
tx["target"] = pd.to_numeric(tx["target"], errors="coerce").astype(int)


In [10]:
beh_nodup = (
    beh
    .sort_values("transdate_dt")
    .drop_duplicates(["cst_dim_id", "transdate_dt"], keep="last")
)

beh_nodup.shape


(8579, 20)

In [11]:
merged = tx.merge(
    beh_nodup,
    on=["cst_dim_id", "transdate_dt"],
    how="left",
    suffixes=("", "_beh")
)

merged.shape, merged["monthly_os_changes"].notna().mean()


((13113, 26), 0.9711736444749485)

In [13]:
merged.to_csv("../data/merged.csv", index=False)
